In [1]:
import wandb
import pprint
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torchvision import datasets, transforms

## Prepare script

In [2]:
wandb.login()

sweep_config = {
    'method': 'bayes'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'fc_layer_size': {
        'values': [128, 256, 512]
        },
    'dropout': {
          'values': [0.3, 0.4, 0.5]
        },
    }

sweep_config['parameters'] = parameters_dict

parameters_dict.update({
    'epochs': {
        'value': 1}
    })

parameters_dict.update({
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
      },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 256,
      }
    })

pprint.pprint(sweep_config)

wandb: Currently logged in as: christopher-marais. Use `wandb login --relogin` to force relogin


{'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'batch_size': {'distribution': 'q_log_uniform_values',
                               'max': 256,
                               'min': 32,
                               'q': 8},
                'dropout': {'values': [0.3, 0.4, 0.5]},
                'epochs': {'value': 1},
                'fc_layer_size': {'values': [128, 256, 512]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.1,
                                  'min': 0},
                'optimizer': {'values': ['adam', 'sgd']}}}


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        loader = build_dataset(config.batch_size)
        network = build_network(config.fc_layer_size, config.dropout)
        optimizer = build_optimizer(network, config.optimizer, config.learning_rate)

        for epoch in range(config.epochs):
            avg_loss = train_epoch(network, loader, optimizer)
            wandb.log({"loss": avg_loss, "epoch": epoch})
    
def build_dataset(batch_size):
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307,), (0.3081,))])
    # download MNIST training dataset
    dataset = datasets.MNIST(".", train=True, download=True,
                             transform=transform)
    sub_dataset = torch.utils.data.Subset(
        dataset, indices=range(0, len(dataset), 5))
    loader = torch.utils.data.DataLoader(sub_dataset, batch_size=batch_size)
    return loader


def build_network(fc_layer_size, dropout):
    network = nn.Sequential(  # fully-connected, single hidden layer
        nn.Flatten(),
        nn.Linear(784, fc_layer_size), nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(fc_layer_size, 10),
        nn.LogSoftmax(dim=1))
    return network.to(device)
        

def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer


def train_epoch(network, loader, optimizer):
    cumu_loss = 0
    for _, (data, target) in enumerate(loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        # ➡ Forward pass
        loss = F.nll_loss(network(data), target)
        cumu_loss += loss.item()
        # ⬅ Backward pass + weight update
        loss.backward()
        optimizer.step()
        wandb.log({"batch loss": loss.item()})
    return cumu_loss / len(loader)

In [4]:
sweep_id = wandb.sweep(sweep_config, project="pytorch-sweeps-demo")

Create sweep with ID: zj1ph8pg
Sweep URL: https://wandb.ai/christopher-marais/pytorch-sweeps-demo/sweeps/zj1ph8pg


In [5]:
wandb.agent(sweep_id, train, count=5)

wandb: Agent Starting Run: t5uevwzw with config:
wandb: 	batch_size: 200
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.014936568577796615
wandb: 	optimizer: sgd


batch loss,██▇▇▆▆▅▅▄▄▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▂▂▂▁▂▁▁▁▂▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,0.32462
epoch,0
loss,0.90414


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: wnllt13o with config:
wandb: 	batch_size: 136
wandb: 	dropout: 0.4
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.032374439475308325
wandb: 	optimizer: adam


batch loss,▂▇█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch,▁
loss,▁
batch loss,1.14381
epoch,0
loss,1.75085


wandb: Agent Starting Run: 6q3xr4tz with config:
wandb: 	batch_size: 152
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.03646306520393821
wandb: 	optimizer: sgd


batch loss,██▇▅▄▄▃▂▃▃▂▃▂▂▂▂▂▂▂▂▃▂▂▂▂▂▂▁▁▂▂▂▂▁▂▁▂▂▁▂
epoch,▁
loss,▁
batch loss,0.33624
epoch,0
loss,0.63818


wandb: Agent Starting Run: 8m72z0mk with config:
wandb: 	batch_size: 128
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.03881718005275972
wandb: 	optimizer: sgd


batch loss,█▇▆▄▃▃▂▂▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▂▂▂▂▂▁▁▁▁▂
epoch,▁
loss,▁
batch loss,0.35074
epoch,0
loss,0.55226


wandb: Agent Starting Run: 0xme9pvl with config:
wandb: 	batch_size: 104
wandb: 	dropout: 0.5
wandb: 	epochs: 1
wandb: 	fc_layer_size: 512
wandb: 	learning_rate: 0.042686762699016736
wandb: 	optimizer: sgd


batch loss,█▇▅▃▃▃▂▂▂▃▁▃▂▃▂▂▂▂▂▂▂▂▁▂▁▂▂▁▁▂▂▂▂▁▂▁▂▁▁▂
epoch,▁
loss,▁
batch loss,0.59546
epoch,0
loss,0.54184
